# TASKS
1. Take only hot elements and hot users. (since there is an option to select a hot user). 
2. Make restrictions by date. 
3. Discard unnecessary features of the item as unnecessary. 
4. It is necessary to enter the surrogate key customer_id/article_id in the table of items/users (init sql).

In [1]:
import os
import pandas as pd

path = "./data"

articles = pd.read_csv(os.path.join(path, "articles.csv"))
customers = pd.read_csv(os.path.join(path, "customers.csv"))
transactions_trains = pd.read_csv(os.path.join(path, "transactions_train.csv"))

In [2]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

Remove columns that will not be used in either the model or the backend.

In [53]:
articles = articles.drop(columns=[
    "product_code",
    "graphical_appearance_no", "graphical_appearance_name",
    "colour_group_code", "colour_group_name",
    "perceived_colour_value_id", "perceived_colour_value_name",
    "perceived_colour_master_id", "perceived_colour_master_name"
])

In [54]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   article_id          105542 non-null  int64 
 1   prod_name           105542 non-null  object
 2   product_type_no     105542 non-null  int64 
 3   product_type_name   105542 non-null  object
 4   product_group_name  105542 non-null  object
 5   department_no       105542 non-null  int64 
 6   department_name     105542 non-null  object
 7   index_code          105542 non-null  object
 8   index_name          105542 non-null  object
 9   index_group_no      105542 non-null  int64 
 10  index_group_name    105542 non-null  object
 11  section_no          105542 non-null  int64 
 12  section_name        105542 non-null  object
 13  garment_group_no    105542 non-null  int64 
 14  garment_group_name  105542 non-null  object
 15  detail_desc         105126 non-null  object
dtypes:

In [55]:
articles["image_id"] = articles["article_id"]

In [56]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355969 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [57]:
transactions_trains.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [58]:
print(f"min date: {transactions_trains['t_dat'].min()}; max date: {transactions_trains['t_dat'].max()}")

min date: 2018-09-20; max date: 2020-09-22


In [59]:
transactions = transactions_trains.copy()

In [60]:
transactions_count = len(transactions)
customers_count = len(transactions["customer_id"].unique())
articles_count = len(transactions["article_id"].unique())

print(f"total transactions: {transactions_count}; unique customers: {customers_count}; unique articles: {articles_count}")

total transactions: 31788324; unique customers: 1362281; unique articles: 104547


### Time boundaries
For simplicity, will take transactions for the last year cause task to design a system that can support long-term operation. That is, replacing the recommendation model, as well as its constant updating, updating data and introducing new functions into the system.
Then:
 - start date: 2020-03-22
 - end date: 2020-09-22

In [61]:
start_date = "2020-03-22"
end_date = "2020-09-22"

mask = (transactions["t_dat"] >= start_date) & (transactions["t_dat"] <= end_date)
cutted_transactions = transactions[mask]

assert cutted_transactions["t_dat"].min() == start_date
assert cutted_transactions["t_dat"].max() == end_date

cutted_transactions_count = len(cutted_transactions)
cutted_customers_count = len(cutted_transactions["customer_id"].unique())
cutted_articles_count = len(cutted_transactions["article_id"].unique())

print(f"total transactions: {cutted_transactions_count}; unique customers: {cutted_customers_count}; unique articles: {cutted_articles_count}")

total transactions: 8185912; unique customers: 748053; unique articles: 51478


### Prepare
We will use a two-stage recommendation model. An important property for the data is the sparseness of the matrix. We will try to numerically increase this characteristic of the data.  
Cause my pc limitation, will immediatly cut off data based on hot users. Hot users are users who have the most transactions. And cut off by hot items.

In [62]:
def sparsity(df, row_name, col_name):
    matrix = df[[row_name, col_name]]
    matrix.loc[0:, "target"] = 1
    matrix = matrix.pivot_table(values="target",
                                index="customer_id",
                                columns="article_id",
                                fill_value=0)

    non_zero_elements = matrix.to_numpy().astype(bool).sum()
    total_elements = matrix.size
    return 1 - non_zero_elements / total_elements

In [63]:
top_users = cutted_transactions.groupby("customer_id").count()
top_users = top_users[top_users["article_id"] > 85].index

hot_items = cutted_transactions.groupby("article_id").count()
hot_items = hot_items[hot_items["customer_id"] > 35].index

cutted_transactions = cutted_transactions[
    cutted_transactions["customer_id"].isin(top_users) &
    cutted_transactions["article_id"].isin(hot_items)
]

print(f"total transactions: {len(cutted_transactions)}; "
      f"unique customers: {len(cutted_transactions['customer_id'].unique())}; "
      f"unique items: {len(cutted_transactions['article_id'].unique())};"
)

# ------------------------------------------------------------------------------- #

print(f"sparsity: {sparsity(cutted_transactions, 'customer_id', 'article_id')}")

total transactions: 440564; unique customers: 3799; unique items: 20588;


C:\Users\user\AppData\Local\Temp\ipykernel_17776\309638371.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



sparsity: 0.9957456491188539


### Visualization
lets check what we got. check hot items categories and other stuff.

In [64]:
import plotly.express as px

In [65]:
detailed_transactions = cutted_transactions.merge(right=articles, how="left", on="article_id")
detailed_transactions = detailed_transactions.merge(right=customers, how="left", on="customer_id")
detailed_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440564 entries, 0 to 440563
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   t_dat                   440564 non-null  object 
 1   customer_id             440564 non-null  object 
 2   article_id              440564 non-null  int64  
 3   price                   440564 non-null  float64
 4   sales_channel_id        440564 non-null  int64  
 5   prod_name               440564 non-null  object 
 6   product_type_no         440564 non-null  int64  
 7   product_type_name       440564 non-null  object 
 8   product_group_name      440564 non-null  object 
 9   department_no           440564 non-null  int64  
 10  department_name         440564 non-null  object 
 11  index_code              440564 non-null  object 
 12  index_name              440564 non-null  object 
 13  index_group_no          440564 non-null  int64  
 14  index_group_name    

In [66]:
def plot_feature(df, feature_name, nbins=None):
    feature = df.groupby(feature_name).count()
    px.histogram(feature, x=feature.index, y="article_id", nbins=nbins).show()

In [67]:
plot_feature(detailed_transactions, "product_type_name")

In [68]:
plot_feature(detailed_transactions, "product_group_name")

In [69]:
detailed_transactions = detailed_transactions[~detailed_transactions["product_group_name"].isin(["Bags", "Items", "Unknown"])]

In [70]:
plot_feature(detailed_transactions, "department_name")

In [71]:
plot_feature(detailed_transactions, "section_name")

In [72]:
plot_feature(detailed_transactions, "index_name")

In [73]:
plot_feature(detailed_transactions, "index_code")

In [74]:
detailed_transactions = detailed_transactions[~detailed_transactions["index_code"].isin(["G", "H", "I", "J"])]

In [75]:
plot_feature(detailed_transactions, "index_group_name")

In [76]:
plot_feature(detailed_transactions, "garment_group_name")

In [77]:
detailed_transactions = detailed_transactions[~detailed_transactions["garment_group_name"].isin(["Unknown"])]

In [78]:
plot_feature(detailed_transactions, "age", 15)

In [79]:
detailed_transactions = detailed_transactions[(19 < detailed_transactions["age"]) & (detailed_transactions["age"] < 45)]

In [80]:
print(
    f"transactions count: {len(detailed_transactions)}\n"
    f"unique items: {len(detailed_transactions['article_id'].unique())};\n"
    f"unique customers: {len(detailed_transactions['customer_id'].unique())}"
)

transactions count: 319568
unique items: 18884;
unique customers: 2828


In [81]:
print(f"sparsity: {sparsity(detailed_transactions, 'customer_id', 'article_id')}")

C:\Users\user\AppData\Local\Temp\ipykernel_17776\309638371.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



sparsity: 0.9954602048927016


### To SQL
load data to sql. make a reservation that the data that will be used for training is the main data of the system.

In [82]:
sql_items = articles[articles["article_id"].isin(detailed_transactions["article_id"].unique())]
sql_customers = customers[customers["customer_id"].isin(detailed_transactions["customer_id"].unique())]
sql_interactions = detailed_transactions

assert len(sql_items) == len(detailed_transactions["article_id"].unique())
assert len(sql_customers) == len(detailed_transactions["customer_id"].unique())
assert len(sql_interactions) == len(detailed_transactions)

Create custom customer_id and article_id using uuid4

In [83]:
import uuid

def asserting_len(df, first_column, second_column):
    assert len(df[first_column].unique()) == len(df[second_column].unique())

sql_customers.loc[0:, "customer_uuid"] = [uuid.uuid4() for _ in range(len(sql_customers))]
sql_items.loc[0:, "article_uuid"] = [uuid.uuid4() for _ in range(len(sql_items))]

asserting_len(sql_customers, "customer_uuid", "customer_id")
asserting_len(sql_items, "article_uuid", "article_id")

C:\Users\user\AppData\Local\Temp\ipykernel_17776\1950152683.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\AppData\Local\Temp\ipykernel_17776\1950152683.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [84]:
sql_interactions["transaction_uuid"] = [uuid.uuid4() for _ in range(len(sql_interactions))]
sql_interactions = sql_interactions.merge(right=sql_customers, how="left", on="customer_id")
sql_interactions = sql_interactions.merge(right=sql_items, how="left", on="article_id")

sql_interactions = sql_interactions[transactions.columns.to_list() + ["transaction_uuid", "customer_uuid", "article_uuid"]]

In [85]:
sql_customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2828 entries, 38 to 1371879
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             2828 non-null   object 
 1   FN                      1448 non-null   float64
 2   Active                  1438 non-null   float64
 3   club_member_status      2821 non-null   object 
 4   fashion_news_frequency  2828 non-null   object 
 5   age                     2828 non-null   float64
 6   postal_code             2828 non-null   object 
 7   customer_uuid           2828 non-null   object 
dtypes: float64(3), object(5)
memory usage: 263.4+ KB


In [86]:
sql_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18884 entries, 1 to 105534
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   article_id          18884 non-null  int64 
 1   prod_name           18884 non-null  object
 2   product_type_no     18884 non-null  int64 
 3   product_type_name   18884 non-null  object
 4   product_group_name  18884 non-null  object
 5   department_no       18884 non-null  int64 
 6   department_name     18884 non-null  object
 7   index_code          18884 non-null  object
 8   index_name          18884 non-null  object
 9   index_group_no      18884 non-null  int64 
 10  index_group_name    18884 non-null  object
 11  section_no          18884 non-null  int64 
 12  section_name        18884 non-null  object
 13  garment_group_no    18884 non-null  int64 
 14  garment_group_name  18884 non-null  object
 15  detail_desc         18865 non-null  object
 16  image_id            18884 

In [87]:
sql_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319568 entries, 0 to 319567
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   t_dat             319568 non-null  object 
 1   customer_id       319568 non-null  object 
 2   article_id        319568 non-null  int64  
 3   price             319568 non-null  float64
 4   sales_channel_id  319568 non-null  int64  
 5   transaction_uuid  319568 non-null  object 
 6   customer_uuid     319568 non-null  object 
 7   article_uuid      319568 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 19.5+ MB


In [88]:
sql_customers = sql_customers[["customer_uuid", "customer_id", "FN", "Active", "club_member_status", "fashion_news_frequency", "age", "postal_code"]]
sql_customers = sql_customers.rename(columns={"FN": "fn", "Active": "active"})

In [89]:
sql_customers["fn"] = sql_customers["fn"].fillna(0).astype(int)

In [90]:
sql_customers["active"] = sql_customers["active"].fillna(0).astype(int)

In [91]:
sql_customers["age"] = sql_customers["age"].fillna(sql_customers["age"].sum() / len(sql_customers)).astype(int)

In [92]:
sql_customers["fashion_news_frequency"] = sql_customers["fashion_news_frequency"].replace("NONE", None)

In [93]:
sql_customers["customer_uuid"] = sql_customers["customer_uuid"].astype(str)

In [94]:
sql_customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2828 entries, 38 to 1371879
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   customer_uuid           2828 non-null   object
 1   customer_id             2828 non-null   object
 2   fn                      2828 non-null   int32 
 3   active                  2828 non-null   int32 
 4   club_member_status      2821 non-null   object
 5   fashion_news_frequency  1453 non-null   object
 6   age                     2828 non-null   int32 
 7   postal_code             2828 non-null   object
dtypes: int32(3), object(5)
memory usage: 230.2+ KB


In [95]:
import psycopg2
from psycopg2.extras import register_uuid

In [96]:
connection = psycopg2.connect("dbname=app_db user=admin password=admin")

In [97]:
def insert(conn, df, table_name):
    """
    Using cursor.mogrify() to build the bulk insert query
    then cursor.execute() to execute the query
    """
    register_uuid()

    print("Start creating tuples...")
    tuples = [tuple(x) for x in df.to_numpy()]

    columns_names = ",".join(list(df.columns))
    placeholders = ",".join(["%s"] * len(df.columns))

    cursor = conn.cursor()
    query  = f"insert into {table_name}({columns_names}) values ({placeholders})"
    print("Starting insert into table...")
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("Execution is done!")
    cursor.close()

In [98]:
sql_customers = sql_customers.drop(columns=["customer_id"])

In [99]:
sql_customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2828 entries, 38 to 1371879
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   customer_uuid           2828 non-null   object
 1   fn                      2828 non-null   int32 
 2   active                  2828 non-null   int32 
 3   club_member_status      2821 non-null   object
 4   fashion_news_frequency  1453 non-null   object
 5   age                     2828 non-null   int32 
 6   postal_code             2828 non-null   object
dtypes: int32(3), object(4)
memory usage: 208.1+ KB


In [100]:
insert(connection, sql_customers, "customers")

Start creating tuples...
Starting insert into table...
Execution is done!


In [101]:
product_group_no = {name: no for no, name in enumerate(sql_items["product_group_name"].unique())}
sql_items["product_group_no"] = sql_items["product_group_name"].apply(lambda x: product_group_no[x])

C:\Users\user\AppData\Local\Temp\ipykernel_17776\1294546139.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [102]:
sql_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18884 entries, 1 to 105534
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   article_id          18884 non-null  int64 
 1   prod_name           18884 non-null  object
 2   product_type_no     18884 non-null  int64 
 3   product_type_name   18884 non-null  object
 4   product_group_name  18884 non-null  object
 5   department_no       18884 non-null  int64 
 6   department_name     18884 non-null  object
 7   index_code          18884 non-null  object
 8   index_name          18884 non-null  object
 9   index_group_no      18884 non-null  int64 
 10  index_group_name    18884 non-null  object
 11  section_no          18884 non-null  int64 
 12  section_name        18884 non-null  object
 13  garment_group_no    18884 non-null  int64 
 14  garment_group_name  18884 non-null  object
 15  detail_desc         18865 non-null  object
 16  image_id            18884 

In [103]:
column_order = [
    "article_uuid", "prod_name",
    "product_type_no", "product_type_name", 
    "product_group_name", "product_group_no",
    "department_no", "department_name",
    "index_code", "index_name",
    "index_group_no", "index_group_name",
    "section_no", "section_name",
    "garment_group_no", "garment_group_name",
    "detail_desc", "image_id"
]

sql_items = sql_items.reindex(columns=column_order)

In [104]:
sql_items["article_uuid"] = sql_items["article_uuid"].astype(str)

In [105]:
sql_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18884 entries, 1 to 105534
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   article_uuid        18884 non-null  object
 1   prod_name           18884 non-null  object
 2   product_type_no     18884 non-null  int64 
 3   product_type_name   18884 non-null  object
 4   product_group_name  18884 non-null  object
 5   product_group_no    18884 non-null  int64 
 6   department_no       18884 non-null  int64 
 7   department_name     18884 non-null  object
 8   index_code          18884 non-null  object
 9   index_name          18884 non-null  object
 10  index_group_no      18884 non-null  int64 
 11  index_group_name    18884 non-null  object
 12  section_no          18884 non-null  int64 
 13  section_name        18884 non-null  object
 14  garment_group_no    18884 non-null  int64 
 15  garment_group_name  18884 non-null  object
 16  detail_desc         18865 

In [106]:
insert(connection, sql_items, "articles")

Start creating tuples...
Starting insert into table...
Execution is done!


In [107]:
column_order = [
    "transaction_uuid", "t_dat", "price", "sales_channel_id", "customer_uuid", "article_uuid"
]

sql_interactions = sql_interactions.reindex(columns=column_order)

In [108]:
sql_interactions["transaction_uuid"] = sql_interactions["transaction_uuid"].astype(str)

In [109]:
sql_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319568 entries, 0 to 319567
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   transaction_uuid  319568 non-null  object 
 1   t_dat             319568 non-null  object 
 2   price             319568 non-null  float64
 3   sales_channel_id  319568 non-null  int64  
 4   customer_uuid     319568 non-null  object 
 5   article_uuid      319568 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 14.6+ MB


In [110]:
insert(connection, sql_interactions, "transactions")

Start creating tuples...
Starting insert into table...
Execution is done!
